In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob

%matplotlib inline

## Pull in SGL Crosswalk
This file (likely not yet finalized) matches sub-ledger data to DATA Act elements

In [20]:
sgl_crosswalk = pd.read_csv(
    'data/other/sgl_account_element_crosswalk.csv',
    dtype = {'code' : np.object}
)

## Create Financial Datasets

### Read JAAMS Files

In [21]:
def read_data(data_dir):
    """returns all .txt files in specified directory as a dict of DataFrames"""
    files = glob.glob('data/{}/*.txt'.format(data_dir))
    df_dict = {}
    for file in files:
        key = file.split('/')[-1][:-4].lower()
        df_dict[key] = pd.read_csv(
            file,
            dtype = {'SEGMENT6' : np.object,
                'PO_HEADER_ID': np.object,
                'PO_LINE_ID': np.object,
                'PO_DISTRIBUTION_ID': np.object,
                'CODE_COMBINATION_ID': np.object,
                'SEGMENT3': np.object,
                'SEGMENT4': np.object,
                'SEGMENT5': np.object,
                'SEGMENT6': np.object,
                'AE_HEADER_ID': np.object,
                'AE_LINE_NUM': np.object,
                'SOURCE_DISTRIBUTION_ID_NUM_1': np.object,
                'INVOICE_ID': np.object,
                'LINE_NUMBER': np.object,
                'INVOICE_LINE_NUMBER': np.object,
                'INVOICE_DISTRIBUTION_ID': np.object,
                'DISTRIBUTION_ID': np.object} 
        )
        rows = len(df_dict[key].index)
        df_dict[key].rename(
            columns=lambda x: '{}.'.format(key) + x.lower(), inplace = True)
        print('Read {} records from {}'.format(
            rows, file))
    return df_dict
jaams = read_data('jaams')
#prism = read_data('prism')

Read 816 records from data/jaams/AP_CHECKS_ALL.txt
Read 1210 records from data/jaams/AP_INVOICE_DISTRIBUTIONS_ALL.txt
Read 1210 records from data/jaams/AP_INVOICE_LINES_ALL.txt
Read 836 records from data/jaams/AP_INVOICE_PAYMENTS_ALL.txt
Read 1209 records from data/jaams/AP_INVOICES_ALL.txt
Read 322 records from data/jaams/AP_SUPPLIER_SITES_ALL.txt
Read 312 records from data/jaams/AP_SUPPLIERS.txt
Read 21 records from data/jaams/AP_TERMS_TL.txt
Read 112 records from data/jaams/FV_FUND_PARAMETERS.txt
Read 48 records from data/jaams/FV_TREASURY_SYMBOLS.txt
Read 544 records from data/jaams/GL_CODE_COMBINATIONS.txt
Read 18984 records from data/jaams/GL_IMPORT_REFERENCES.txt
Read 1888 records from data/jaams/GL_JE_BATCHES.txt
Read 5894 records from data/jaams/GL_JE_HEADERS.txt
Read 19486 records from data/jaams/GL_JE_LINES.txt
Read 259 records from data/jaams/HR_LOCATIONS_ALL.txt
Read 30 records from data/jaams/MTL_CATEGORIES_B.txt
Read 579 records from data/jaams/PO_DISTRIBUTIONS_ALL.txt
R

### Create Subledger DataFrame

In [22]:
xladist = jaams['xla_distribution_links']
xlalines = jaams['xla_ae_lines']
glcode = jaams['gl_code_combinations']
print ('starting: xlalines rows = {}'.format(len(xlalines.index)))
sgl = pd.merge(
    xladist,
    xlalines,
    left_on = ['xla_distribution_links.ae_header_id', 'xla_distribution_links.ae_line_num'],
    right_on = ['xla_ae_lines.ae_header_id', 'xla_ae_lines.ae_line_num']
)
print ('after xladist merge: {} rows'.format(len(sgl.index)))
#merge in code combination lookup for SGL account
sgl = pd.merge(
    sgl,
    glcode,
    left_on = ['xla_ae_lines.code_combination_id'],
    right_on = ['gl_code_combinations.code_combination_id']
)
print ('after glcode merge: {} rows'.format(len(sgl.index)))
sgl = sgl.rename(
    columns = {'gl_code_combinations.segment6' : 'gl_code_combinations.sgl_account'}
)
sgl = pd.merge(
    sgl,
    sgl_crosswalk,
    left_on = ['gl_code_combinations.sgl_account'],
    right_on = ['sgl_account']
    )
print ('after sgl crosswalk merge: {} rows'.format(len(sgl.index)))
sgl = sgl[['xla_ae_lines.ae_header_id', 
   'xla_ae_lines.ae_line_num',
   'xla_ae_lines.accounted_dr',
   'xla_ae_lines.accounted_cr',
   'xla_distribution_links.source_distribution_id_num_1',
   'xla_distribution_links.source_distribution_type',
   'gl_code_combinations.sgl_account',
    'data_act_element',
    'code_description']]

starting: xlalines rows = 18766
after xladist merge: 18987 rows
after glcode merge: 18987 rows
after sgl crosswalk merge: 8760 rows


### Create Purchase Orders DataFrame

In [23]:
# join po data
po = pd.DataFrame()
poh = jaams['po_headers_all'][['po_headers_all.po_header_id', 'po_headers_all.segment1']]
pol = jaams['po_lines_all'][['po_lines_all.po_header_id', 'po_lines_all.po_line_id', 'po_lines_all.item_description', 'po_lines_all.quantity','po_lines_all.unit_price']]
poll = jaams['po_line_locations_all'][['po_line_locations_all.po_line_id']]
pod = jaams['po_distributions_all'][['po_distributions_all.po_distribution_id', 'po_distributions_all.po_header_id', 'po_distributions_all.po_line_id', 'po_distributions_all.code_combination_id', 'po_distributions_all.attribute10','po_distributions_all.attribute11', 'po_distributions_all.attribute_category', 'po_distributions_all.quantity_billed', 'po_distributions_all.req_distribution_id']]
fv_fund = jaams['fv_fund_parameters']
fv_tas = jaams['fv_treasury_symbols']
print ('starting: headers_all rows = {}'.format(len(poh.index)))
po = pd.merge(
    poh,
    pol,
    left_on = ['po_headers_all.po_header_id'],
    right_on = ['po_lines_all.po_header_id']
    )
print ('after po_lines_all merge: {} rows'.format(len(po.index)))
po = pd.merge(
    po,
    poll,
    left_on = ['po_lines_all.po_line_id'],
    right_on = ['po_line_locations_all.po_line_id']
)
print ('after po_line_locations merge: {} rows'.format(len(po.index)))
po = pd.merge(
    po,
    pod,
    left_on = ['po_lines_all.po_line_id'],
    right_on = ['po_distributions_all.po_line_id']
)
print ('after po_distributions_all merge: {} rows'.format(len(po.index)))
po.to_csv('data/jaams_po_before_gl_code_combinations_merge.csv', index = False)
po = pd.merge(
    po,
    glcode,
    left_on = ['po_distributions_all.code_combination_id'],
    right_on = ['gl_code_combinations.code_combination_id']
)
print ('after gl_code_combinations merge: {} rows'.format(len(po.index)))
po = pd.merge(
    po,
    sgl[sgl['xla_distribution_links.source_distribution_type'] == 'PO_DISTRIBUTIONS_ALL'],
    how = 'left',
    left_on = ['po_distributions_all.po_distribution_id'],
    right_on = ['xla_distribution_links.source_distribution_id_num_1']
)
print ('after sgl merge: {} rows'.format(len(po.index)))
#grab TAS
po = pd.merge(
    po,
    fv_fund,
    left_on = 'gl_code_combinations.segment2',
    right_on = 'fv_fund_parameters.fund_value'
    )
po = pd.merge(
    po,
    fv_tas,
    left_on = 'fv_fund_parameters.treasury_symbol_id',
    right_on = 'fv_treasury_symbols.treasury_symbol_id')
print ('after fv_fund merge: {} rows'.format(len(po.index)))

#tidy up
po = po[['po_headers_all.po_header_id',
    'po_lines_all.po_line_id',
    'po_distributions_all.po_distribution_id',
    'po_distributions_all.req_distribution_id',
    'po_headers_all.segment1', #award number
    'po_lines_all.item_description',
    'po_lines_all.quantity',
    'po_lines_all.unit_price',
    'po_distributions_all.quantity_billed',
    'po_distributions_all.code_combination_id',
    'po_distributions_all.attribute10', #period of performance start dt
    'po_distributions_all.attribute11', #period of performance end dt
    'gl_code_combinations.segment3', #funding office
    'gl_code_combinations.segment4', #program activity
    'gl_code_combinations.segment5', #object class
    'fv_treasury_symbols.treasury_symbol', #tas
    'xla_ae_lines.ae_header_id',
    'xla_ae_lines.ae_line_num',
    'xla_ae_lines.accounted_dr',
    'xla_ae_lines.accounted_cr',
    'gl_code_combinations.sgl_account',
    'data_act_element',
    'code_description'        
    ]]

#combine debits and credits into a single columns
def get_value(row):
    credit = row['xla_ae_lines.accounted_cr']
    debit = row['xla_ae_lines.accounted_dr']
    if pd.isnull(debit):
        return credit
    else:
        return debit * -1
po['xla_ae_lines.accounted_amt'] = po.apply(lambda row: get_value(row),axis=1)
po.to_csv('data/jaams_po.csv', index = False)

#aggregate
po_agg = pd.pivot_table(po,
    index=['po_headers_all.po_header_id',
    'po_lines_all.po_line_id',
    'po_distributions_all.po_distribution_id',
    'po_distributions_all.req_distribution_id',
    'po_headers_all.segment1',
    'po_lines_all.item_description',
    'xla_ae_lines.ae_header_id',
    'xla_ae_lines.ae_line_num',
    'po_distributions_all.attribute10', #period of performance start dt
    'po_distributions_all.attribute11', #period of performance end dt
    'gl_code_combinations.segment3', #funding office
    'gl_code_combinations.segment4', #program activity
    'gl_code_combinations.segment5',
    'fv_treasury_symbols.treasury_symbol'],
    columns = ['data_act_element'],
    values = ['xla_ae_lines.accounted_amt']
)
po_agg = po_agg.groupby(
    level=['po_headers_all.po_header_id',
    'po_lines_all.po_line_id',
    'po_distributions_all.po_distribution_id',
    'po_distributions_all.req_distribution_id',
    'po_headers_all.segment1',
    'po_lines_all.item_description',
    'po_distributions_all.attribute10', #period of performance start dt
    'po_distributions_all.attribute11', #period of performance end dt
    'gl_code_combinations.segment3', #funding office
    'gl_code_combinations.segment4', #program activity
    'gl_code_combinations.segment5',
    'fv_treasury_symbols.treasury_symbol']
).sum()
po_agg.columns = po_agg.columns.get_level_values(1)
po_agg.to_csv('data/jaams_po_pivot.csv')

starting: headers_all rows = 579
after po_lines_all merge: 1197 rows
after po_line_locations merge: 1197 rows
after po_distributions_all merge: 1197 rows
after gl_code_combinations merge: 1145 rows
after sgl merge: 2054 rows
after fv_fund merge: 2054 rows


### Create Invoice Distributions DataFrame

In [24]:
aida = jaams['ap_invoice_distributions_all']
print ('starting: invoice_distributions rows = {}'.format(len(aida.index)))
invd = pd.merge(
    aida,
    sgl[sgl['xla_distribution_links.source_distribution_type'] == 'AP_INV_DIST'],
    left_on = ['ap_invoice_distributions_all.invoice_distribution_id'],
    right_on = ['xla_distribution_links.source_distribution_id_num_1']
    )
print ('after sgl merge: {} rows'.format(len(invd.index)))

#tidy up
invd = invd[[
    'ap_invoice_distributions_all.po_distribution_id',
    'xla_ae_lines.ae_header_id',
    'xla_ae_lines.ae_line_num',
    'xla_ae_lines.accounted_dr',
    'xla_ae_lines.accounted_cr',
    'data_act_element'     
    ]]

#combine debits and credits into a single columns
def get_value(row):
    credit = row['xla_ae_lines.accounted_cr']
    debit = row['xla_ae_lines.accounted_dr']
    if pd.isnull(debit):
        return credit
    else:
        return debit * -1
invd['xla_ae_lines.accounted_amt'] = invd.apply(lambda row: get_value(row),axis=1)
invd.to_csv('data/jaams_invd.csv', index = False)

#aggregate
invd_agg = pd.pivot_table(invd,
    index=[
        'ap_invoice_distributions_all.po_distribution_id',
        'xla_ae_lines.ae_header_id',
        'xla_ae_lines.ae_line_num',
    ],
    columns = ['data_act_element'],
    values = ['xla_ae_lines.accounted_amt']
)
invd_agg = invd_agg.groupby(
    level=['ap_invoice_distributions_all.po_distribution_id']
    ).sum()
invd_agg.columns = invd_agg.columns.get_level_values(1)
invd_agg.reset_index(inplace = True)
po_agg.reset_index(inplace = True)
invd_agg.to_csv('data/jaams_invd_pivot.csv')

# join this to po distributions
po_agg = pd.merge(
    po_agg,
    invd_agg,
    left_on = 'po_distributions_all.po_distribution_id',
    right_on = 'ap_invoice_distributions_all.po_distribution_id',
    how = 'left',
    suffixes = ('_po', '_inv')  
)
print ('after po merge: {} rows'.format(len(po_agg.index)))

# math!
po_agg['obligated_amt'] = po_agg['account_obligation_amt_po'] + po_agg['account_obligation_amt_inv']
po_agg = po_agg.rename(columns = {'account_outlay_amt' : 'outlay_amt'})
#po_agg.drop(po_agg.columns[[10,11,12,13]],axis = 1, inplace = True)

po_agg.to_csv('data/jaams_subledger_agg.csv', index = False)

starting: invoice_distributions rows = 1210
after sgl merge: 1736 rows
after po merge: 439 rows
